In [2]:
import pandas as pd
from gliner import GLiNER
import spacy
from tqdm.notebook import tqdm
import pickle

In [3]:
custom_spacy_config = { "gliner_model": "urchade/gliner_small-v2.1",
                            "chunk_size": 250,
                            "labels": ["people","company","organization","location","date","money"],
                            "style": "dep",
                            "threshold": 0.5,
                            "map_location": "mps"}
nlp = spacy.blank("en")
nlp.add_pipe("gliner_spacy", config=custom_spacy_config)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/611M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/Users/megpatakota/Library/Caches/pypoetry/virtualenvs/graph-6ebOGMnL-py3.12/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
df = pd.read_csv('Global-Cases-Export-2024-09-25.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')
# split at jurisdictions to jurisdictions_country, jurisdictions_region, jurisdictions_type given the > separator. Eg: 'Australia>New South Wales>Land and Environment Court'
# Note: not all rows have all 3 values
df[['jurisdictions_country', 'jurisdictions_region', 'jurisdictions_type']] = df['jurisdictions'].str.split('>',n=2, expand=True)
df.head()
df = df.sample(10)

In [5]:
def split_text_to_chunks(text, chunk_size=250):
    return [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]


chunk_nodes = []
chunk_id = 1
for index, row in df.iterrows():
    text_chunks = split_text_to_chunks(row["summary"], 250)
    doc_id = row["id"]
    for chunk in text_chunks:
        chunk_nodes.append({"id": chunk_id, "doc_id": doc_id, "text": chunk})
        chunk_id += 1

# for chunk in chunk_nodes:
#     print(chunk)
chunk_nodes


def get_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append(
            {
                "text": ent.text,
                "start_char": ent.start_char,
                "end_char": ent.end_char,
                "label": ent.label_,
            }
        )
    return entities


entity_nodes = []
entity_id = 1
for chunk in tqdm(chunk_nodes):
    entities = get_entities(chunk["text"])
    for entity in entities:
        entity_nodes.append(
            {
                "id": entity_id,
                "chunk_id": chunk["id"],
                "text": entity["text"],
                "start_char": entity["start_char"],
                "end_char": entity["end_char"],
                "label": entity["label"],
            }
        )
        entity_id += 1

  0%|          | 0/85 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
entity_nodes_sample = [node for node in entity_nodes if node['chunk_id'] == 1]
chunk_nodes_sample = [node for node in chunk_nodes if node['id'] == 1]
chunk_nodes_sample,entity_nodes_sample


([{'id': 1,
   'doc_id': 14616,
   'text': 'On September 9, 2021, the youth plaintiffs Henrike Cremer, Jan Luca Samuel Salis, and Friedrich Hirschmann, supported by environmental organization Deutsche Umwelthilfe (DUH; Environmental Action Germany), brought a constitutional claim against the s'}],
 [{'id': 1,
   'chunk_id': 1,
   'text': 'September 9, 2021',
   'start_char': 3,
   'end_char': 20,
   'label': 'date'},
  {'id': 2,
   'chunk_id': 1,
   'text': 'Henrike Cremer',
   'start_char': 43,
   'end_char': 57,
   'label': 'people'},
  {'id': 3,
   'chunk_id': 1,
   'text': 'Jan Luca Samuel Salis',
   'start_char': 59,
   'end_char': 80,
   'label': 'people'},
  {'id': 4,
   'chunk_id': 1,
   'text': 'Friedrich Hirschmann',
   'start_char': 86,
   'end_char': 106,
   'label': 'people'},
  {'id': 5,
   'chunk_id': 1,
   'text': 'Deutsche Umwelthilfe',
   'start_char': 148,
   'end_char': 168,
   'label': 'organization'},
  {'id': 6,
   'chunk_id': 1,
   'text': 'DUH',
   'start_char'

In [8]:
# save the chunk and entities to pickle
pickle.dump(chunk_nodes, open("./graph_data/chunk_nodes_sample.pkl", "wb"))
pickle.dump(entity_nodes, open("./graph_data/entity_nodes_sample.pkl", "wb"))